# UniTVelo benchmark on cell cycle data

Notebook benchmarks velocity, latent time inference, and cross boundary correctness using UniTVelo on cell cycle data.

In [ ]:
import os

import pandas as pd

import anndata as ad
import scvelo as scv
import unitvelo as utv
from cellrank.kernels import VelocityKernel

from rgv_tools import DATA_DIR
from rgv_tools.benchmarking import get_time_correlation

## General settings

In [ ]:
scv.settings.verbosity = 3

In [ ]:
velo_config = utv.config.Configuration()
velo_config.R2_ADJUST = True
velo_config.IROOT = None
velo_config.FIT_OPTION = "1"
velo_config.AGENES_R2 = 1
velo_config.GPU = -1

In [ ]:
os.environ["TF_USE_LEGACY_KERAS"] = "True"

## Constants

In [ ]:
DATASET = "cell_cycle"

In [ ]:
STATE_TRANSITIONS = [("G1", "S"), ("S", "G2M")]

In [ ]:
SAVE_DATA = True
if SAVE_DATA:
    (DATA_DIR / DATASET / "results").mkdir(parents=True, exist_ok=True)

## Data loading

In [ ]:
adata = ad.io.read_h5ad(DATA_DIR / DATASET / "processed" / "adata_processed.h5ad")
adata

## Velocity pipeline

In [ ]:
adata.obs["cluster"] = "0"
adata = utv.run_model(adata, label="cluster", config_file=velo_config)

In [ ]:
time_correlation = [get_time_correlation(ground_truth=adata.obs["fucci_time"], estimated=adata.obs["latent_time"])]

In [ ]:
scv.tl.velocity_graph(adata, vkey="velocity", n_jobs=1)
scv.tl.velocity_confidence(adata, vkey="velocity")

## Cross-boundary correctness

In [ ]:
vk = VelocityKernel(adata, vkey="velocity").compute_transition_matrix()

cluster_key = "phase"
rep = "X_pca"

score_df = []
for source, target in STATE_TRANSITIONS:
    cbc = vk.cbc(source=source, target=target, cluster_key=cluster_key, rep=rep)

    score_df.append(
        pd.DataFrame(
            {
                "State transition": [f"{source} - {target}"] * len(cbc),
                "CBC": cbc,
            }
        )
    )
score_df = pd.concat(score_df)

## Data saving

In [ ]:
if SAVE_DATA:
    pd.DataFrame({"time": time_correlation}, index=adata.obs_names).to_parquet(
        path=DATA_DIR / DATASET / "results" / "unitvelo_correlation.parquet"
    )
    adata.obs[["velocity_confidence"]].to_parquet(path=DATA_DIR / DATASET / "results" / "unitvelo_confidence.parquet")
    score_df.to_parquet(path=DATA_DIR / DATASET / "results" / "unitvelo_cbc.parquet")